BIGRAM MODEL

In a sequence of text, bigrams are pairs of consecutive words or tokens. Bigrams allow us to see which words commonly co-occur within a given dataset

('You', 'are')
('are', 'learning')
('learning', 'from')

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250


cpu


In [2]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
chars=sorted(set(text))
#print(chars)
print(len(chars))
vocab_size = len(chars)

80


In [3]:
#Tokenization
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# encoded=(encode('hello'))
# print(encoded)
# print(decode(encode('hello')))
data=torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [4]:
n=int(0.8*len(data))
train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
   
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[60, 68, 67, 58,  1, 74, 69,  1],
        [58, 71, 75, 58, 57,  1, 73, 61],
        [79, 58, 57,  1, 68, 67, 58,  0],
        [73,  1, 54, 71, 58,  1, 73, 61]])
targets:
tensor([[68, 67, 58,  1, 74, 69,  1, 73],
        [71, 75, 58, 57,  1, 73, 61, 58],
        [58, 57,  1, 68, 67, 58,  0, 61],
        [ 1, 54, 71, 58,  1, 73, 61, 68]])


Block size is the length of each sequence and Batch size is how many of these we are actually doing at the same time

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



0D.3*mQ9Pi
fRy?AFkBPuSJJ;QIDvcx4Xo!RCmm9rkUy;AR
!TGmMkO.1KCUQPKEg0FoGLdnQnm"?X7tGD)bXPtu[!RJ?6SpcblK-7Y1RR0h0bY'cMGcAfJE"_:J)mi;hBu7E_AKB (:C!,-tVyUOnmf:GWAHcqsYLCmtUq]*10T6w_.M3]_]6:XkKJ(fzjpZ.aIL0O?VIjwZmtQ3vs-_Z'M"U"Abdv;3F"fluf3q)!
uSORUWz'J?0fRKRRw0CUq:,-7vuKA:KLn'JJwex530yCrQ;Q,jCuoLbcR[ls?g[[ fl&w(D0Aax('JMp.8eluvSR
3 7J, 7.kICU(5a:70WLGpSYI'Ex5qZj)]!M3'MT6cJVg-SvXzoeY'0Apx'MrBaIOPRA(di?sPi.kBe)k8Znq4O_"9f8[fzJlGs8ORk5a2 aMe;H)Q37.MGsPiKpAV]ZoG[U"_'rrgktwc6.al&wEIjR,P" B"rn(k)aE[jyUZCT r(


In [7]:
block_size = 8
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    content=x[:t+1]
    target=y[t]
    print("When input is",content, "target is",target)

When input is tensor([1]) target is tensor(1)
When input is tensor([1, 1]) target is tensor(28)
When input is tensor([ 1,  1, 28]) target is tensor(39)
When input is tensor([ 1,  1, 28, 39]) target is tensor(42)
When input is tensor([ 1,  1, 28, 39, 42]) target is tensor(39)
When input is tensor([ 1,  1, 28, 39, 42, 39]) target is tensor(44)
When input is tensor([ 1,  1, 28, 39, 42, 39, 44]) target is tensor(32)
When input is tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) target is tensor(49)


In [8]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.883, val loss: 4.880
step: 250, train loss: 4.796, val loss: 4.800
step: 500, train loss: 4.764, val loss: 4.746
step: 750, train loss: 4.679, val loss: 4.672


KeyboardInterrupt: 

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())


print(generated_chars)